# Menu categorization
In this notebook I manually categorize the menu items found in `menu_data.sqlite3`.

Menu items fall into the following categories:

* beverage
* dessert
* condiment
* side order
* entree

Items end up in these categories according to my own idiosyncratic categorization.

In [1]:
import sqlite3
import json
import os

database_filename = "menu_data.sqlite3"

## Create second (temporary) database to hold menu categories
Instead of writing logic to check if a `menu_category` table exists in `menu_data.sqlite3` *OR* a `menu_category` column exists in the `menu_data` table, I'm just going to create a new, temporary, database file to hold the category data. I will create this file in a similar fashion to what I did in `download_data.ipynb`.

In [2]:
# Get `menu_id`s from `menu_data.sqlite3` to populate temporary database.
with sqlite3.connect(database_filename) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT item_id FROM menu_data")
    item_ids = cursor.fetchall()

# Abstractions for temporary category database
category_database_filename = "menu_categories.sqlite3"

# I assume that if the file `menu_categories.sqlite3` already exists, the schema and data exist as well.
db_is_new = not os.path.exists(category_database_filename)

with sqlite3.connect(category_database_filename) as conn:
    if db_is_new:
        conn.execute("CREATE TABLE item_categories (item_id TEXT PRIMARY KEY NOT NULL, item_category text)")
        cursor = conn.cursor()
        cursor.executemany("INSERT INTO item_categories (item_id) VALUES (?)", item_ids)

## Dump non-categorized `item_name`s to a file
In this way I can delete the items which belong to a particular category and use `set` operations to recover the list of deleted items.

In [11]:
# I've abstracted the JOIN command up here because i wanted to break it up over multiple lines.
join_cmd = """
SELECT menu_data.menu_data.item_name
    FROM menu_data.menu_data JOIN menu_categories.item_categories 
        ON menu_data.menu_data.item_id = menu_categories.item_categories.item_id
    WHERE menu_categories.item_categories.item_category ISNULL
"""

# I'm not sure of a nice looking way to do this.
with sqlite3.connect(":memory:") as conn:
    # Ugly. Would like to abstract.
    conn.execute("ATTACH DATABASE 'menu_data.sqlite3' as 'menu_data'")
    conn.execute("ATTACH DATABASE 'menu_categories.sqlite3' as 'menu_categories'")
    cursor = conn.cursor()    
    cursor.execute(join_cmd)

    item_names = cursor.fetchall()
    
item_names = [item_name[0] for item_name in item_names]
item_names.sort()
    
with open("item_names.dat", "w") as f:
    f.write(json.dumps(item_names, indent=4, separators=(',', ': ')))

At this point I will open the `item_names.dat` file in a text editor and remove each line that lists a menu item belonging to a particular category (beverage, side, etc.). Once I've deleted all the item names of a particular category, I will save the file.

Next, I will open the saved file and recover the deleted item names using `set` operations.

In [20]:
with open("item_names.dat", "r") as f:
    truncated_item_names = json.load(f)
    
categorized_item_names = list(set(item_names) - set(truncated_item_names))
categorized_item_names.sort()

Finally, I will `INSERT` the `categorized_item_names` into the `menu_category` database.

In [22]:
# Add code

In [36]:
# conn = sqlite3.connect(database_filename)
# conn.execute("ATTACH DATABASE ? as db2", (category_database_filename, ))